## 1. 일반 FashionMNIST 학습

In [1]:
# 라이브러리 불러오기
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# loader 함수 정의
def get_mnist_loaders(train_batch_size, test_batch_size):
    """Get MNIST data loaders"""
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=train_batch_size, shuffle=True)
    
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True)

    return train_loader, test_loader

In [3]:
# 모델 정의
class Net(nn.Module):
    def __init__(self, activation):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        self.activation = activation 

    def forward(self, x):
        x = self.activation(self.conv1(x))
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.activation(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [4]:
# train, test 정의
def train(log_interval, model, train_loader, optimizer, epoch, device):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        loss = F.nll_loss(output, target.to(device))
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))           

def test(model, test_loader, device):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data.to(device))
            test_loss += F.nll_loss(output, target.to(device), reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.to(device).view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)

In [5]:
# 전체 시스템 정의
def train_mnist():
    cfg = { 'device' : "cuda" if torch.cuda.is_available() else "cpu",
          'train_batch_size' : 64,
          'test_batch_size' : 1000,
          'n_epochs' : 1,
          'seed' : 0,
          'log_interval' : 100,
          'save_model' : False,
          'lr' : 0.001,
          'momentum': 0.5,
          'optimizer': optim.SGD,
          'activation': F.relu
          }

    torch.manual_seed(cfg['seed'])
    train_loader, test_loader = get_mnist_loaders(cfg['train_batch_size'], cfg['test_batch_size'])
    model = Net(cfg['activation']).to(cfg['device'])
    optimizer = cfg['optimizer'](model.parameters(), lr=cfg['lr'])
    for epoch in range(1, cfg['n_epochs'] + 1):
        train(cfg['log_interval'], model, train_loader, optimizer, epoch, cfg['device'])
        test_accuracy = test(model, test_loader, cfg['device'])

    if cfg['save_model']:
        torch.save(model.state_dict(), "C:\\Users\\Bang\\JupyterProjects\\Pytorch_GridSearch_Example\\model\\mnist_cnn.pt")

    return test_accuracy

if __name__ == '__main__':
    train_mnist()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298800
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.216094
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.097683
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.019460
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.774781
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.463632
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.138829
Train Epoch: 1 [44800/60000 (75%)]	Loss: 1.096267
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.095197
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.576627


## 2. GridSearch 학습

In [6]:
# 패키지 설치
!pip install optuna

In [7]:
# 라이브러리 불러오기(GridSearch)
import optuna

In [8]:
# 전체 시스템 정의
def train_mnist(trial):
    cfg = { 'device' : "cuda" if torch.cuda.is_available() else "cpu",
        'train_batch_size' : 64,
        'test_batch_size' : 1000,
        'n_epochs' : trial.suggest_int('n_epochs', 3, 5, 1),
        'seed' : 0,
        'log_interval' : 100,
        'save_model' : False,
        'lr' : trial.suggest_loguniform('lr', 1e-3, 1e-2),  
        'momentum' : trial.suggest_uniform('momentum', 0.4, 0.99),
        'optimizer': trial.suggest_categorical('optimizer',[optim.SGD, optim.RMSprop]),
        'activation': F.relu
        }

    torch.manual_seed(cfg['seed'])
    train_loader, test_loader = get_mnist_loaders(cfg['train_batch_size'], cfg['test_batch_size'])
    model = Net(cfg['activation']).to(cfg['device'])
    optimizer = cfg['optimizer'](model.parameters(), lr=cfg['lr'])
    for epoch in range(1, cfg['n_epochs'] + 1):
        train(cfg['log_interval'], model, train_loader, optimizer, epoch, cfg['device'])
        test_accuracy = test(model, test_loader, cfg['device'])

    if cfg['save_model']:
        torch.save(model.state_dict(), "C:\\Users\\Bang\\JupyterProjects\\Pytorch_GridSearch_Example\\model\\mnist_cnn.pt")

    return test_accuracy

In [9]:
# GridSearch 정의
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='maximize')

# GridSearch 학습
study.optimize(train_mnist, n_trials=5) # study.optimize(train_mnist, n_trials=20, direction='maximize')

[I 2022-11-28 14:53:47,437] A new study created in memory with name: no-name-0f1f6355-b1fe-4099-b71d-c15d824902f6
C:\Users\Bang\AppData\Local\Temp\ipykernel_19752\2320783044.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lr' : trial.suggest_loguniform('lr', 1e-3, 1e-2),
C:\Users\Bang\AppData\Local\Temp\ipykernel_19752\2320783044.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'momentum' : trial.suggest_uniform('momentum', 0.4, 0.99),
C:\Users\Bang\Anaconda\envs\BangEnv\lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent sto

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298800
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.153982
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.933691
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.704710
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.305684
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.889822
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.659555
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.765362
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.857396
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.381847
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.472006
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.705421
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.406385
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.414987
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.454198
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.445403
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.486965
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.292097
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.355914
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.397281
Train Epoch:

[W 2022-11-28 14:54:21,934] Trial 0 failed because of the following error: The value None could not be cast to float.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298800
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.916815
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.738410
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.773309
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.665731
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.338953
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.211691
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.436145
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.459279
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.185565
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.234768
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.482173
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.165351
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.241864
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.274767
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.288420
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.251635
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.161490
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.217981
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.231815
Train Epoch:

[W 2022-11-28 14:55:08,315] Trial 1 failed because of the following error: The value None could not be cast to float.


...

[W 2022-11-28 14:55:54,551] Trial 2 failed because of the following error: The value None could not be cast to float.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298800
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.822247
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.635373
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.683305
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.444544
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.231782
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.271380
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.745299
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.179065
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.239266
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.234127
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.421748
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.116661
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.229756
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.190156
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.173884
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.083465
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.049521
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.130986
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.196971
Train Epoch:

[W 2022-11-28 14:56:40,865] Trial 3 failed because of the following error: The value None could not be cast to float.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298800
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.664164
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.061702
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.993751
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.787368
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.423050
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.299174
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.484947
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.550201
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.225208
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.262270
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.555984
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.215195
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.272887
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.322814
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.319662
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.312370
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.191914
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.272106
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.285112
Train Epoch:

[W 2022-11-28 14:57:38,719] Trial 4 failed because of the following error: The value None could not be cast to float.


In [10]:
# 라이브러리 불러오기(joblib)
import joblib

# GridSearch 학습 저장
joblib.dump(study, 'C:\\Users\\Bang\\JupyterProjects\\Pytorch_GridSearch_Example\\GridSearch\\mnist_optuna.pkl')

['C:\\Users\\Bang\\JupyterProjects\\Pytorch_GridSearch_Example\\GridSearch\\mnist_optuna.pkl']

In [11]:
study = joblib.load('C:\\Users\\Bang\\JupyterProjects\\Pytorch_GridSearch_Example\\GridSearch\\mnist_optuna.pkl')
df = study.trials_dataframe().drop(['duration', 'state','datetime_start','datetime_complete'], axis=1)
# 최적의 파라미터를 확인하기 위해서 study.best_trial및 study.best_params도 사용할 수 있다.
# 예를 들어 파라미터 간의 관계를 확인하기 위해 plot_parallel_coordinates(study) 이라는 명령어를 사용하여 아래와 같은 결과를 얻을 수 있다. (이 예시에서는 lr과 momentum이 파라미터)
# 다른 방법으로 contour plot을 이용할 수도 있다. 이 결과는 plot_contour(study) 를 통해 얻을 수 있다.
# 또한 slice_plot(study) 을 호출하여 slice plot을 만들 수 있다. 이는 각 파라미터에 대해 개별적인 최적의 부분 공간이 어디에 위치하는지 이해하는 데 도움이 될 수 있다.
# 마지막으로 study history를 시각화 하기 위해 plot_optimization_history(study) 을 이용할 수도 있다.

# 실제 학습에 사용되는 함수를 수정한다. (예시에서 train_mnist 함수가 이에 해당한다. ) 이 함수는 아래의 조건을 충족해야 한다.
# 1. 평가 지표가 될 점수를 return 해야 한다. 이 함수 안에서 모델의 파라미터를 정해준다. (trial.suggest_.. 함수 이용). 이 함수는 trial을 매개변수로 받아야 한다.
# 2. study.optimize 를 실행한다. 1에서 정의한 함수를 파라미터로 넘겨준다. 예시: study.optimize(train_mnist, n_trials=20, direction='maximize')
df.head(3)

,number,value,params_lr,params_momentum,params_n_epochs,params_optimizer
0,0,None,0.001541,0.487012,3,<class 'torch.optim.sgd.SGD'>
1,1,None,0.007495,0.881453,4,<class 'torch.optim.sgd.SGD'>
2,2,None,0.008898,0.930849,4,<class 'torch.optim.rmsprop.RMSprop'>
